In [1]:
import pandas as pd
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Loading data
- df1 = human data
- df2 = synthetic data

In [6]:
xls = pd.ExcelFile('FinalTopicAnnotations_shareable.xlsx')
df1 = pd.read_excel(xls, 'top words')#human data topics
df2 = pd.read_excel(xls, 'synthetic_fullword')#synthetic topics

We manually group together the topics into the topics in UMD-ODH
Left side = human Right side = synthetic

In [9]:
topic_dict = {
    'family': ([2], [18]),
    'work':([5,17], [3, 24]),
    'health':([1, 3], [4]),
    'finance':([22], [6]),
    'relationship':([10, 19], [21]),
    'school':([11, 13], [12]),
    'news and social media':([7], [11, 17]),
    'unemployment':([17], [6]),
    'stress': ([0], [10])
}

topic_words = {}

Get the tokens for each overarching groups

In [11]:
df1_vocab = {}
df2_vocab = {}

for k, v in topic_dict.items():
    df1_topics = df1.iloc[v[0], :]['words'].values.sum().replace(' ', '').split(',')
    df2_topics = df2.iloc[v[1], :]['words'].values.sum().replace(' ', '').split(',')
    if k not in df1_vocab:
        df1_vocab[k] = []
        df2_vocab[k] = []
    df1_vocab[k] = df1_topics
    df2_vocab[k] = df2_topics

family ([2], [18])
work ([5, 17], [3, 24])
health ([1, 3], [4])
finance ([22], [6])
relationship ([10, 19], [21])
school ([11, 13], [12])
news and social media ([7], [11, 17])
unemployment ([17], [6])
stress ([0], [10])


Loading Glove model

In [222]:
import random
from tqdm import tqdm
import torch
import numpy as np
import torch.nn.functional as F

embeddings = {}
with open("../../../../local/glove.6B.300d.txt", "r", encoding="utf-8") as file_p:
    for line in tqdm(file_p):
        values = line.split()
        word = values[0]
        vector = torch.Tensor(np.asarray(values[1:], "float32"))
        embeddings[word] = vector

-0.09240163

# Similarity of topics
Obtaining the avg. cosine similarity by taking the mean of the embeddings for human and synthetic topics and calculating the cosine similarity

In [20]:

for k in topic_dict.keys():
    # print(k)
    human = df1_vocab[k]        
    synth = df2_vocab[k]
    synth_embs = [
        x for x in [embeddings.get(token) for token in synth] if x is not None
    ]
    human_embs = [
        x for x in [embeddings.get(token) for token in human] if x is not None
    ]

    synth_embs = torch.stack(synth_embs)
    human_embs = torch.stack(human_embs)

    synth_mean = synth_embs.mean(dim=0)
    human_mean = human_embs.mean(dim=0)

    sim_score = F.cosine_similarity(synth_mean, human_mean, dim=0)

    print('topic:', k, ' score ', sim_score)


topic: family  score  tensor(0.7889)
topic: work  score  tensor(0.9010)
topic: health  score  tensor(0.8567)
topic: finance  score  tensor(0.9352)
topic: relationship  score  tensor(0.9233)
topic: school  score  tensor(0.9070)
topic: news and social media  score  tensor(0.8434)
topic: unemployment  score  tensor(0.8780)
topic: stress  score  tensor(0.8848)


Calculating the baseline by finding the avg cosine similarity for a random pair of topics

In [21]:
rands = []
for i in range(100):
    rand_words1 = random.sample(list(embeddings.keys()), k=30)
    rand_words2 = random.sample(list(embeddings.keys()), k=30)

    rand_embs1 = torch.stack([embeddings.get(tok) for tok in rand_words1])
    rand_embs2 = torch.stack([embeddings.get(tok) for tok in rand_words2])

    rand_mean1 = rand_embs1.mean(dim=0)
    rand_mean2 = rand_embs2.mean(dim=0)

    sim_score = F.cosine_similarity(rand_mean1, rand_mean2, dim=0)
    rands.append(sim_score)

print(np.mean(rands))

0.75299853